# Input Filenames Here

`season_data_file` is the data file from which to create features (could be only fraction of regular season in future). `team_features_output_file` is the filename where the features for each team will be saved.

In [27]:
season_data_file = 'data/RegularSeasonDetailedResults.csv' 
team_features_output_file = 'data/team_features.csv'

In [28]:
import pandas as pd
import numpy as np

season_df = pd.read_csv(season_data_file)
teams_df = pd.read_csv('data/Teams.csv')
team_name = dict(zip(teams_df['Team_Id'].values, teams_df['Team_Name'].values))
team_id = dict(zip(teams_df['Team_Name'].values, teams_df['Team_Id'].values))

# Create dictionaries for boolean masks
seasons = season_df['Season'].unique()
in_season = dict(zip(seasons, [season_df['Season'] == i for i in seasons]))

team_ids = teams_df['Team_Id'].unique()
is_winner = dict(zip(team_ids, [season_df['Wteam'] == i for i in team_ids]))
is_loser = dict(zip(team_ids, [season_df['Lteam'] == i for i in team_ids]))
did_play = dict(zip(team_ids, [(season_df['Wteam'] == i) | (season_df['Lteam'] == i) for i in team_ids]))# winner or loser

is_valid = season_df['Wteam'] != 0

def avg(feature): # pass string for feature (without W or L to get a function back that calculates average for the season)
    def avg_feat(team):
        f = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'W'+feature], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'L'+feature]])
        return np.round(f.mean(), decimals=2)
    avg_feat.__name__ = 'avg_'+feature
    return avg_feat

def std(feature): # pass string for feature (without W or L to get a function back that calculates stdev for the season)
    def std_feat(team):
        f = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'W'+feature], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'L'+feature]])
        return np.round(f.std(), decimals=2)
    std_feat.__name__ = 'std_'+feature
    return std_feat


# ADD NEW TEAM FEATURES HERE (at bottom of cell below)

Add your new features specific to a team here, see tutorial at bottom of notebook.  You can create features that generate simple averages or standard deviations of features in the detailed data using the avg and std functions above.  What you add here to `teamfList` are the features that will be appended to the data file.

In [29]:
def wins(team):  
    return season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid].shape[0]

# Four factors
def efg(team):
    fgm = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfgm'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfgm']])
    fgm3 = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfgm3'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfgm3']])
    fga = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfga'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfga']])
    efg = (fgm+0.5*fgm3)/fga
    return np.round(efg.mean(), decimals=2)

def to_p(team):
    to = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wto'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lto']])
    fga = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfga'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfga']])
    o_r = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wor'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lor']])
    fta = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfta'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfta']])
    possessions = 0.96*(fga-o_r-to+0.475*fta)
    to_p = to/possessions
    return np.round(to_p.mean(), decimals=2)

def orp(team):
    o_r = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wor'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lor']])
    dr_op = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Ldr'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wdr']])
    orp = o_r/(o_r + dr_op)
    return np.round(orp.mean(), decimals=2)

def ftr(team):
    fta = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfta'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfta']])
    fga = pd.concat([season_df.loc[is_winner[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Wfga'], season_df.loc[is_loser[team['Team_Id']] & in_season[team['Season']] & is_valid, 'Lfga']])
    ftr = fta/fga
    return np.round(ftr.mean(), decimals=2)

# Append your new function to teamfList below!

In [30]:
teamfList = []
#teamfList.append(to_p)
#teamfList.append(orp)
#teamfList.append(wins)
#for f in ['fgm', 'fga', 'fgm3', 'fga3', 'ftm', 'fta', 'or', 'dr', 'ast', 'to', 'stl', 'blk', 'pf']:
#    teamfList.append(avg(f))
#    teamfList.append(std(f))
print("Team Features that will be added = {0}".format([f.__name__ for f in teamfList]))

Team Features that will be added = []


In [31]:
#USE THESE TWO LINES TO REGENERATE ALL THE TEAM FEATURES
#team_features_df = pd.DataFrame([[team_ID, season] for team_ID in team_ids for season in seasons], columns = ['Team_Id', 'Season'])
#team_features_df['team_name'] = team_features_df['Team_Id'].map(team_name)

#USE THIS LINE TO ADD TO THE EXISTING TEAM FEATURES
team_features_df = pd.read_csv(team_features_output_file, index_col=0)

for feature in teamfList:
    team_features_df[feature.__name__] = team_features_df.apply(feature, axis=1)

team_features_df = team_features_df.replace(np.nan, 0)
team_features_df.to_csv(team_features_output_file, encoding='ascii')
team_features_df.tail()

,Team_Id,Season,team_name,wins,avg_fgm,std_fgm,avg_fga,std_fga,avg_fgm3,std_fgm3,...,avg_stl,std_stl,avg_blk,std_blk,avg_pf,std_pf,ftr,efg,to_p,orp
4727,1464,2011,Youngstown St,7,24.71,3.79,59.75,6.74,7.86,2.95,...,6.21,2.13,4.82,2.63,20.18,5.22,0.30,0.48,0.29,0.26
4728,1464,2012,Youngstown St,14,23.79,3.54,55.31,6.39,8.52,3.00,...,6.28,2.53,5.31,2.42,16.55,4.49,0.31,0.51,0.30,0.28
4729,1464,2013,Youngstown St,14,23.90,4.05,56.34,8.07,6.66,2.64,...,6.34,2.86,5.31,2.12,17.72,4.15,0.37,0.49,0.30,0.26
4730,1464,2014,Youngstown St,12,25.38,4.09,55.90,7.67,7.59,2.35,...,6.38,2.78,2.48,1.72,19.03,4.34,0.41,0.52,0.26,0.24
4731,1464,2015,Youngstown St,8,25.03,3.80,56.66,7.12,6.62,2.16,...,5.90,2.76,1.93,1.67,18.62,5.52,0.37,0.51,0.28,0.24


# Input Filenames for Generating Features for the Games

`training_data_file` is the data file we want to use as training data (could include part of regular season in addition to tourney in future). `team_features_input_file` is the input file we want to use for the team features (generated above--default is `team_features_output_file`, i.e. what was used for output above).  `training_data_output_file` is the filename for the training data with all the games.

In [32]:
training_data_file = 'data/data.csv' # use data/TourneyDetailedResults.csv to regenerate features
team_features_input_file = team_features_output_file
training_data_output_file = 'data/data.csv'

# Randomizing the Winning Team and Adding Features

We first randomly chooses the winner to be team 1 or team 0.  The result is stored in `tourney_df('Winner')` and the ID corresponding to team0 and team1 are in `tourney_df('team0')` and `tourney_df('team1')`, respectively.  We also add all the features in `data/team_features.csv` for team0 and team1.

In [33]:
tourney_df = pd.read_csv(training_data_file, index_col=0) # take off index_col if regenerating with TourneyDetailedResults

Ngames = tourney_df.shape[0]
print("{0} tournament games in the dataset".format(Ngames))

np.random.seed(0) # seed so we always get same answer
tourney_df['team0Win'] = np.random.randint(2,size=Ngames)

def team0(game):
    if game['team0Win'] == 1:
        return game['Wteam']
    else:
        return game['Lteam']

def team1(game):
    if game['team0Win'] == 1:
        return game['Lteam']
    else:
        return game['Wteam']

team_features_df = pd.read_csv(team_features_input_file, index_col=0)
seasons = tourney_df['Season'].unique()
in_season = dict(zip(seasons, [team_features_df['Season'] == i for i in seasons]))

def feature01(feature):
    def mod(game):
        f0 = team_features_df.loc[(team_features_df['Team_Id'] == game['team0']) & in_season[game['Season']], feature.__name__].values[0]
        f1 = team_features_df.loc[(team_features_df['Team_Id'] == game['team1']) & in_season[game['Season']], feature.__name__].values[0]
        return pd.Series({feature.__name__+'0':f0, feature.__name__+'1':f1})
    return mod

847 tournament games in the dataset


In [39]:
print(tourney_df.columns[0:40])

Index([u'Season', u'Daynum', u'Wteam', u'Wscore', u'Lteam', u'Lscore', u'Wloc',
       u'Numot', u'Wfgm', u'Wfga', u'Wfgm3', u'Wfga3', u'Wftm', u'Wfta',
       u'Wor', u'Wdr', u'Wast', u'Wto', u'Wstl', u'Wblk', u'Wpf', u'Lfgm',
       u'Lfga', u'Lfgm3', u'Lfga3', u'Lftm', u'Lfta', u'Lor', u'Ldr', u'Last',
       u'Lto', u'Lstl', u'Lblk', u'Lpf', u'team0Win', u'team0', u'team1',
       u'wins0', u'wins1', u'avg_fgm0'],
      dtype='object')


In [51]:
# UNCOMMENT THESE LINES IF YOU WANT TO REGENERATE THE TRAINING DATA FILE FROM SCRATCH
#for feature in [team0, team1]:
#    tourney_df[feature.__name__] = tourney_df.apply(feature, axis=1)

# first add all the features for individual teams
for feature in teamfList:
    tourney_df = pd.concat([tourney_df, tourney_df.apply(feature01(feature), axis=1)], axis=1)
    
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,SAG1,MOR1,BOB1,WOL1,WLK1,POM1,DOL1,COL1,RTH1,RPI1
0,2003,134,1421,92,1411,84,N,1,32,69,...,251,277,236,220,237,273,258,213,250,186
1,2003,136,1112,80,1436,51,N,0,31,66,...,158,187,158,177,153,144,156,144,182,144
2,2003,136,1113,84,1272,71,N,0,31,59,...,24,23,22,19,21,20,21,19,19,26
3,2003,136,1141,79,1166,73,N,0,29,53,...,48,50,49,49,32,50,37,35,55,44
4,2003,136,1143,76,1301,74,N,1,27,64,...,46,31,47,47,51,48,49,56,42,53


# Add game features here

In addition to having features for each team separately, e.g., avg field goals made, we might try features for how two teams relate to one another, i.e., a feature that requires knowledge of the specific two teams in a game.  Add them in the cell below and update gamefList in the cell below that:

In [52]:
systems = ['SAG', 'MOR', 'BOB', 'WOL', 'WLK', 'POM', 'DOL', 'COL', 'RTH', 'RPI']

def ratings(df):
    for system in systems:
        for i in ['0', '1']:
            df[system+'_rating'+i] = 100.-4*np.log(1.+df[system+i])-df[system+i]/22.
            
def diffs(df):
    for system in systems:
        df['d'+system] = df[system+'0'] - df[system+'1']
        df['d'+system+'_rating'] = df[system+'_rating0'] - df[system+'_rating1']

In [37]:
massey_df = pd.read_csv('data/massey_ordinals_2003-2015.csv')# Create dictionaries for boolean masks

seasons = massey_df['season'].unique()
in_massey_season = dict(zip(seasons, [massey_df['season'] == i for i in seasons]))

massey_day_nums = massey_df['rating_day_num'].unique()
is_massey_day_num = dict(zip(massey_day_nums, [massey_df['rating_day_num'] == i for i in massey_day_nums]))

massey_teams = massey_df['team'].unique()
is_massey_team = dict(zip(massey_teams, [massey_df['team'] == i for i in massey_teams]))

massey_systems = massey_df['sys_name'].unique()
is_massey_system = dict(zip(massey_systems, [massey_df['sys_name'] == i for i in massey_systems]))

def massey(game): # add here any features specific to a game, i.e., how to teams interact
    try:
        daynum = game['Daynum']
    except:
        daynum = 134 # submission file games don't have daynum, but are all after day 133 (last day before tourney)
        
    latest_day = np.max(massey_day_nums[massey_day_nums < daynum]) # only take ordinals from dates that happened before the game, and take latest one of those
    ordinals0 = [np.float64(massey_df.loc[in_massey_season[game['Season']] & is_massey_system[systems[i]] & is_massey_day_num[latest_day] & is_massey_team[game['team0']], 'orank'].values[0]) for i in range(len(systems))]
    ordinals1 = [np.float64(massey_df.loc[in_massey_season[game['Season']] & is_massey_system[systems[i]] & is_massey_day_num[latest_day] & is_massey_team[game['team1']], 'orank'].values[0]) for i in range(len(systems))]    
    f0 = pd.Series(ordinals0, index=[systems[i]+'0' for i in range(len(systems))])
    f1 = pd.Series(ordinals1, index=[systems[i]+'1' for i in range(len(systems))])
    return pd.concat([f0,f1])

# Append your new function to gamefList below!

In [22]:
gamefList=[] # features specific to a pair of teams (rather than to an individual team, e.g., seed difference, but not e.g. team seed)
#gamefList.append(massey)
print("Game features to add = {0}".format([f.__name__ for f in gamefList]))

Game features added = []


In [53]:
# now add features specific to a game (pair of teams)
for feature in gamefList:
    tourney_df = pd.concat([tourney_df, tourney_df.apply(feature, axis=1)], axis=1)

#ratings(tourney_df)
#diffs(tourney_df)

tourney_df.to_csv(training_data_output_file, encoding='ascii')
tourney_df.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,dPOM,dPOM_rating,dDOL,dDOL_rating,dCOL,dCOL_rating,dRTH,dRTH_rating,dRPI,dRPI_rating
0,2003,134,1421,92,1411,84,N,1,32,69,...,-5,0.300940,-19,1.168393,-1,0.064190,6,-0.367220,31,-2.022637
1,2003,136,1112,80,1436,51,N,0,31,66,...,-141,20.770848,-154,22.830534,-142,21.967031,-180,24.625314,-142,21.967031
2,2003,136,1113,84,1272,71,N,0,31,59,...,12,-2.353395,27,-4.430384,19,-3.534954,16,-3.078419,6,-1.075410
3,2003,136,1141,79,1166,73,N,0,29,53,...,-23,3.443939,-19,3.636225,-17,3.329047,-33,5.059430,-21,3.468980
4,2003,136,1143,76,1301,74,N,1,27,64,...,-8,1.076629,-24,3.706615,-28,3.975749,-12,1.854306,-20,2.759585


# Generate Features for Submission Data

The cells below take the games Kaggle wants us to submit, and generates the corresponding features for them like we do above for the training data.  `submission_data_output_file` is the name of the output file to write the submission data for our model to load and predict.

In [32]:
import pandas as pd
import numpy as np
submission_data_with_features_file = 'data/2016SampleSubmissionWithFeatures.csv'

In [33]:
def tourneylike(game):
    sag0 = np.float(game['SAG0'])
    sag1 = np.float(game['SAG1'])
    if np.isnan(sag0) or np.isnan(sag1):
        return -999
    day = 134.
    return day/133. - sag0/347 - sag1/347

In [34]:
teamfList = []
gamefList = []

In [35]:
print("Adding following Team Features to {0}: {1}".format(submission_data_with_features_file, [f.__name__ for f in teamfList]))
print("Adding following Game Features to {0}: {1}".format(submission_data_with_features_file, [f.__name__ for f in gamefList]))

Adding following Team Features to data/2016SampleSubmissionWithFeatures.csv: []
Adding following Game Features to data/2016SampleSubmissionWithFeatures.csv: []


In [36]:
import numpy as np
import pandas as pd

class FeatureAdder(object):
    def __init__(self, df):
        self.df = df
        
    def feature01(self, feature):
        def mod(game):
            f0 = team_features_df.loc[(team_features_df['Team_Id'] == game['team0']) & in_season[game['Season']], feature.__name__].values[0]
            f1 = team_features_df.loc[(team_features_df['Team_Id'] == game['team1']) & in_season[game['Season']], feature.__name__].values[0]
            return pd.Series({feature.__name__+'0':f0, feature.__name__+'1':f1})
        return mod
     
    def massey(self):
        self.massey_init()
        self.df = pd.concat([self.df, self.df.apply(self.massey_game, axis=1)], axis=1)

    def massey_init(self):
        self.massey_df = pd.read_csv('data/MasseyOrdinals2016ThruDay133_58systems.csv')
        
        # Create dictionaries for boolean masks

        self.massey_seasons = self.massey_df['season'].unique()
        self.in_massey_season = dict(zip(self.massey_seasons, [self.massey_df['season'] == i for i in self.massey_seasons]))

        self.massey_day_nums = self.massey_df['rating_day_num'].unique()
        self.is_massey_day_num = dict(zip(self.massey_day_nums, [self.massey_df['rating_day_num'] == i for i in self.massey_day_nums]))

        self.massey_teams = self.massey_df['team'].unique()
        self.is_massey_team = dict(zip(self.massey_teams, [self.massey_df['team'] == i for i in self.massey_teams]))

        self.massey_systems = self.massey_df['sys_name'].unique()
        self.is_massey_system = dict(zip(self.massey_systems, [self.massey_df['sys_name'] == i for i in self.massey_systems]))

    def massey_game(self, game): # add here any features specific to a game, i.e., how do teams interact       
        try:
            daynum = game['Daynum']
        except:
            daynum = 134 # submission file games don't have daynum, but are all after day 133 (last day before tourney)

        latest_day = np.max(self.massey_day_nums[self.massey_day_nums < daynum]) # only take ordinals from dates that happened before the game, and take latest one of those
        ordinals0 = [self.getOrdinal(game['Season'], self.massey_systems[i], latest_day, game['team0']) for i in range(len(self.massey_systems))]
        ordinals1 = [self.getOrdinal(game['Season'], self.massey_systems[i], latest_day, game['team1']) for i in range(len(self.massey_systems))]
        f0 = pd.Series(ordinals0, index=[self.massey_systems[i]+'0' for i in range(len(self.massey_systems))])   
        f1 = pd.Series(ordinals1, index=[self.massey_systems[i]+'1' for i in range(len(self.massey_systems))])   
        return pd.concat([f0,f1])
    
    def getOrdinal(self, season, system, day, team):
        try:
            ordinal = np.float64(self.massey_df.loc[self.in_massey_season[season] & self.is_massey_system[system] & self.is_massey_day_num[day] & self.is_massey_team[team], 'orank'].values[0])
        except IndexError:
            ordinal = np.nan
        return ordinal

def add_features(df):
    dfa = FeatureAdder(df)
    dfa.massey()
    return dfa.df

In [10]:
sample_df = pd.read_csv(submission_data_with_features_file, index_col=0) #should be SampleSubmission.csv if generating from scratch (remove index_col=0 in that case)

In [16]:
dfnew = add_features(sample_df)

In [18]:
dfnew.to_csv(submission_data_with_features_file)

In [12]:
df = pd.read_csv('data/MasseyOrdinals2016ThruDay133_58systems.csv')
df.head()

,season,rating_day_num,sys_name,team,orank
0,2016,16,ACU,1101,340
1,2016,16,ACU,1102,233
2,2016,16,ACU,1103,71
3,2016,16,ACU,1104,111
4,2016,16,ACU,1105,303


In [37]:
sample_df = pd.read_csv(submission_data_with_features_file, index_col=0) #should be SampleSubmission.csv if generating from scratch (remove index_col=0 in that case)

def get_season(Id):
    return np.int64(Id.split("_")[0])
def get_team0(Id):
    return np.int64(Id.split("_")[1])
def get_team1(Id):
    return np.int64(Id.split("_")[2])

# UNCOMMENT THESE LINES IF YOU WANT TO REGENERATE THE SUBMISSION FEATURES FROM SCRATCH
#sample_df['Season'] = sample_df['Id'].apply(get_season)
#sample_df['team0'] = sample_df['Id'].apply(get_team0)
#sample_df['team1'] = sample_df['Id'].apply(get_team1)

# first add all the features for individual teams
for feature in teamfList:
    sample_df = pd.concat([sample_df, sample_df.apply(feature01(feature), axis=1)], axis=1)
    
# now add features specific to a game (pair of teams)
for feature in gamefList:
    #sample_df = pd.concat([sample_df, sample_df.apply(feature, axis=1)], axis=1)
    sample_df[feature.__name__] = sample_df.apply(feature, axis=1)
#ratings(sample_df)
#diffs(sample_df)

sample_df.to_csv(submission_data_with_features_file, encoding='ascii')
sample_df.head()

,Id,Pred,Season,team0,team1,ACU0,AP0,BBT0,BWE0,CPA0,...,WOB1,ARG1,FAS1,MCL1,DC1,STH1,CTL1,RPI1,HKB1,tourneylike
0,2016_1112_1114,0.5,2016,1112,1114,19.0,17.0,14.0,14.0,NaN,...,49.0,36.0,53.0,88.0,54.0,49.0,NaN,46.0,55.0,0.768326
1,2016_1112_1122,0.5,2016,1112,1122,19.0,17.0,14.0,14.0,NaN,...,204.0,197.0,209.0,186.0,187.0,209.0,NaN,189.0,220.0,0.344695
2,2016_1112_1124,0.5,2016,1112,1124,19.0,17.0,14.0,14.0,NaN,...,18.0,25.0,23.0,23.0,18.0,25.0,NaN,25.0,23.0,0.903772
3,2016_1112_1138,0.5,2016,1112,1138,19.0,17.0,14.0,14.0,NaN,...,108.0,123.0,118.0,98.0,102.0,130.0,NaN,91.0,134.0,0.589651
4,2016_1112_1139,0.5,2016,1112,1139,19.0,17.0,14.0,14.0,NaN,...,32.0,32.0,27.0,30.0,36.0,35.0,NaN,56.0,31.0,0.886481


# Tutorial for adding a new feature

All functions to populate a feature should take a game, i.e., a row of `tourney_df`,  

In [11]:
game = tourney_df.loc[0]
print(game)

Season       2003
Daynum        134
Wteam        1421
Wscore         92
Lteam        1411
Lscore         84
Wloc            N
Numot           1
Wfgm           32
Wfga           69
Wfgm3          11
Wfga3          29
Wftm           17
Wfta           26
Wor            14
Wdr            30
Wast           17
Wto            12
Wstl            5
Wblk            3
Wpf            22
Lfgm           29
Lfga           67
Lfgm3          12
Lfga3          31
Lftm           14
Lfta           31
Lor            17
Ldr            28
Last           16
            ...  
std_fta0      8.2
std_fta1     9.02
avg_or0     13.17
avg_or1     12.28
std_or0      4.86
std_or1      3.73
avg_dr0      24.8
avg_dr1     23.17
std_dr0      5.55
std_dr1      5.44
avg_ast0     14.2
avg_ast1    13.03
std_ast0      4.4
std_ast1     4.79
avg_to0     15.23
avg_to1     16.21
std_to0      4.03
std_to1         5
avg_stl0     6.43
avg_stl1     7.07
std_stl0     2.31
std_stl1     3.14
avg_blk0     2.23
avg_blk1        3
std_blk0  

If it's a feature specific to a team, name it 0 or 1, and access team0's and/or team1's ID using the corresponding column.  `team_name` is a dictionary that takes a team ID and returns the name, and `team_id` is a dictionary that takes a team name and returns the ID.  You can use these to match up data you find elsewhere.

The function should return the value for your new feature.  As a simple example, to make a feature with the team's name, 

In [18]:
def name0(game):
    ID = game['team0']
    
    # Examples of using the dictionaries
    name = team_name[ID] 
    print(team_id[name]) 
    
    return name

name0(tourney_df.loc[0])

1411


'TX Southern'

# More Realistic Example

I had to look at different pandas examples/tutorials, but this might get you started.  For `wins0`, I wanted to be able to return the number of games `team0` won in the regular season.  What we actually have is a list of games, each of which have a winning team.  So we need to find all the games where `team0` was the winner, and add them all up.  So we start with the column that has all the IDs of the winning teams for each game.

In [19]:
wteam = season_df['Wteam']
print(wteam.head())

0    1104
1    1272
2    1266
3    1296
4    1400
Name: Wteam, dtype: int64


Now we make a boolean array that has the same length as `wteam` above, with `True` if the winning team is `team0` (which we take to be 1106 in this example):

In [20]:
games_where_1106_won = season_df['Wteam'] == 1106
print(games_where_1106_won.head())

0    False
1    False
2    False
3    False
4    False
Name: Wteam, dtype: bool


In [21]:
games_1106_won = wteam[games_where_1106_won]
print(games_1106_won.head())

613     1106
1095    1106
1584    1106
1714    1106
1873    1106
Name: Wteam, dtype: int64


But this has games for all seasons mixed together, so we have to choose the right season.  Say the game for which we're trying to make the feature happend in 2012.  Then we need another mask like we used above to separate the games where 1106 won, but this time for the season:

In [22]:
in_2012 = season_df['Season'] == 2012
print(in_2012.head())

0    False
1    False
2    False
3    False
4    False
Name: Season, dtype: bool


We can put boolean masks together like this:

In [23]:
games_1106_won_in_2012 = wteam[games_where_1106_won & in_2012]
print(games_1106_won_in_2012.head())

45628    1106
46766    1106
46845    1106
47078    1106
47248    1106
Name: Wteam, dtype: int64


For convenience, I've made a dictionary `in_season` that takes a year, and gives you the corresponding boolean mask for that year.  Using that notation, and finally also using the `is_valid` mask, which ensures that the game is valid (WTeam != 0), we have 

In [24]:
games_1106_won_in_2012 = wteam[games_where_1106_won & in_season[2012] & is_valid]
print(games_1106_won_in_2012.head())

45628    1106
46766    1106
46845    1106
47078    1106
47248    1106
Name: Wteam, dtype: int64


We can now get the length of the final array of filtered values using `shape`, and taking `[0]`, which gives the number of rows.  A minor final detail is that all these new variables make copies of slices of the dataframes, so it's slightly more efficient to do things without making copies with the `loc` function, which explains the one-liners in the wins0 and wins1 functions above: 

In [25]:
print(games_1106_won_in_2012.shape[0])
print(season_df.loc[(season_df['Wteam'] == 1106) & in_season[2012] & is_valid].shape[0])

10
10


# Additional Features

Someother features to consider on the historic record of one Seed versus another. And the spread betting record. Here is an example of generate feature from data/TourneySeedBetting.csv

**TBD: find resource for 2016 league**

In [26]:
tourney_seeddf = pd.read_csv('data/TourneySeedBetting.csv')
tourney_seeddf.head()

### Wseed gives the Seed for the winning team, Lseed gives the Seed for the losing team. 
### meanProb gives the Probability of the team with the smaller seed wins, numProb 
### gives the number of games this probability was calculated based on
### spreadbet give the spreadbetting from covers on the winning team versus losing team.

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wseed,Lseed,meanProb,numProb,spreadbet
0,2011,134,1155,70,1412,52,N,0,12,12,0.50,1,0
1,2011,134,1421,81,1114,77,N,1,16,16,0.50,1,0
2,2011,135,1427,70,1106,61,N,0,16,16,0.50,1,0
3,2011,135,1433,59,1425,46,N,0,11,11,0.50,1,0
4,2011,136,1139,60,1330,58,N,0,8,9,0.51,124,1


Here is an example to generate **meanprob** feature in the table. Since we randomize the teams in the traning set, we need to reasign the meanprob based on which team has a smaller seed. 
**TBD**: get a meanprob feature table for the 2016 games. 

In [27]:
def meanprob(game):
    
    game_seeddf=tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]
    prob=game_seeddf['meanProb'].iloc[0]
    Wseed=game_seeddf['Wseed'].iloc[0]
    Lseed=game_seeddf['Lseed'].iloc[0]
    
    #this does not need to be rescaled because it is already between 0 and 1 
    if game['team0']==0:
        if Wseed < Lseed:
            return 1-prob
        else:
            return prob
    else:
        if Wseed < Lseed:
            return prob
        else:
            return 1-prob


Example for generate the **spreadbet** feature. 

In [1]:
def spreadbet(game):
    
    points=tourney_seeddf[(tourney_seeddf['Wteam'] == game['Wteam']) & (tourney_seeddf['Lteam']==game['Lteam']) & (tourney_seeddf['Season']==game['Season'])]['spreadbet'].iloc[0]
    #The spreadbets in original table was defined as the bets of the Wteam versus Lteam. 
    #We rescale spreadbet by dividing everynumber by 10.
    if game['team0']==game['Wteam']:
        return (points)/10.
    else:
        return -(points)/10.

In [2]:
game = tourney_df[tourney_df['Season']==2011].iloc[0]
print game.head()
meanprob(game)

NameError: name 'tourney_df' is not defined